# EDS 220 Fall 2022
# Mapping and map projections with Google Earth Engine

Google Earth Engine is an extremely powerful tool for visualizing environmental data, making it an extremely popular choice in many contexts. However, the combination of Google Earth Engine with Cartopy is even more powerful, combining the on-the-fly ability of GEE to access and analyze data with Cartopy's strengths in map reprojection and generation of production-quality graphics.

To facilitate the use of Cartopy, we'll use the `cartoee` package. Cartoee functions as a 'go-between' for Earth Engine and Cartopy, by requesting images from Earth Engine and passing them through to Cartopy. In that way, the data can be reprojected as desired using Cartopy without the user having to download all the data from Earth Engine. Because of the utility of this approach, cartoee has been incorporated into `geemap` and can be called from that package such that it can be referred to on its own.

First, we import the relevant packages and initialize the Earth Engine environment:

In [1]:
# Import packages
import ee
import geemap
import pandas as pd
import cartopy
import cartopy.crs as ccrs
from geemap import cartoee
import matplotlib.pyplot as plt

In [2]:
# Authenticate and initialize GEE
#ee.Authenticate()
ee.Initialize()

Next, let's load some data. To get some variety from the Landsat examples we used in previous class sessions, let's pick a marine example!

The following example uses the _ocean color_ data collected by NASA's MODIS satellite:
https://developers.google.com/earth-engine/datasets/catalog/NASA_OCEANDATA_MODIS-Aqua_L3SMI#description

This product is derived from reflectance at the ocean surface in wavelength bands strongly affected by chlorophyll absorption; in this way, we can get an indication of how much photosynthesis by _phytoplankton_ (algae) is happening in the surface ocean.

Note that since ocean color is a derived product calculated from reflectance, it is considered a _Level 3_ data product (see description link above).

Let's load the set of MODIS data into a Google Earth Engine `ImageCollection`:

In [39]:
# Load the MODIS ocean color data


The metadata for this image collection can be accessed through the Google Earth Engine web interface:
https://developers.google.com/earth-engine/datasets/catalog/NASA_OCEANDATA_MODIS-Aqua_L3SMI#bands

or we can get a sense for things by looking at a single image and using the `getInfo` function. To do the latter, we first need to select a single image (-not- an ImageCollection) to let us display that information. 

In [59]:
# Extract the first image in the MODIS image collection


# Display metadata for first MODIS image


As we did in the `Landsat_colormaps_GEE` notebook, it's desirable to reduce the size of the dataset in order to create something that can actually be plotted as a single map.

To accomplish this, let's do the following:
- get rid of cloudy days by filtering on the `CLOUD_COVER` attribute of the image collection;
- select a date range and take the time average over that range

In [41]:
# Use filter to extract all "non-cloudy" images: ones with less than 20% cloud cover


Now we need to figure out a relevant region to display. Here I've chosen one that gives us a sense of variability in chlorophyll through the __California Current__, which is our local current system off the Santa Barbara coast and a good example of a _western boundary current_. The California Current is an extremely _productive_ marine ecosystem because of the amount of _upwelling_ of cold subsurface ocean waters, making it rich in nutrients and therefore home to lots of different marine ecosystems.

To filter the image collection, we set up a central point using `ee.Geometry.Point`, and then define a `buffer` region around that point. Here I've chosen 1000km but you can change this if you like!

In [42]:
# Define the location of interest as a point near Los Angeles, California

# Define the region: everything within 1000km of LA


Once the region is defined, we can then use it to extract the portions of the image collection intersecting with that region, similar to what we did with Landsat for Santa Barbara. This can be accomplished using `filterBounds`, and providing the region object we created above as an argument:

In [43]:
# Filter to region of interest

# Command to extract all data over appropriate time, perform temporal averaging


Now we have the data we need to plot! But we still have to supply some _visualization parameters_ in order to make everything display properly. Below I create an example of the data structure that Google Earth Engine needs to interpret these visualizations. 

Important parameters:
- the band name ("chlor_a") - note that there is only ONE band here, instead of the three we were using to create RGB images with Landsat information;
- minimum and maximum values to consider when assigning colors to the color bar later

In [46]:
# Visualization parameters


When the plot is generated, we can also use all of the functionality of Cartopy to set the plot parameters. Below, I've drawn in the coast outlines using `ax.coastlines()`, and used the `ax.gridlines` module to draw semitransparent gray dashed lat/lon grid lines:

In [58]:
# Create figure object

# create a new Plate-Carree projection 

# plot the result with cartoee 

# Define the lon/lat extent of the plot
  

# Draw in coastlines


# Draw grid lines

# Set parameters for grid lines


# Set plot title

# Add color bar

# Display plot


Now let's compare this with the visualization we'd have gotten if we used the interactive "base map" provided by Google Earth Engine. 

To do this, we need to specify a new set of visualization parameters, since we need another parameter called the color _palette_. The two codes in the palette correspond to the colors of the minimum and maximum values, and GEE will then assign colors ranging between the two specified values to all the data in between.

In [54]:
# Visualization parameters


Now we can draw the basemap and add the MODIS layer to it!

In [57]:
# Define parameters for GEE base map


# Draw map


In [56]:
# Add layer to the basemap
